# Teste Amostral da Linha 010

Código feito por Igor Laltuf, ago/23

---

## Objetivo

Este notebook tem como objetivo realizar o teste amostral da linha 010 com dados da primeira quinzena de janeiro de 2023.

## Etapas
1) Comparar dados do gabarito enviado pelos consórcios com os dados da solução proposta pela DTDI.

2) Verificar se os veículos não identificados na solução operaram nos dias e horários apontados no gabarito, com base em dados de GPS e de conformidade das viagens.

## Conclusão

Ao considerar um intervalo de 10 minutos para mais ou para menos entre os horários de partida, foram encontradas 699 das 706 viagens apresentadas no gabarito (94,75% das viagens do gabarito).

Após verificar as sete viagens não identificadas na solução acima, concluiu-se que:

- Não foram encontrados dados das quatro viagens do veículo A72037 na linha 010, mas foram encontrados sinais de GPS em horários compatíveis com duas das quatro viagens, operando na linha 007.

- No caso do veículo A72085, na viagem do dia 15/01/2023 foram encontrados sinais de GPS para o mesmo veículo, dia e serviço (mas não foram encontrados dados para este dia na tabela de conformidade). Quanto a viagem do dia 04/01/2023, foram identificados sinais de GPS apenas em horário próximo ao término da viagem.

- Para o veículo A72151, foi encontrada uma viagem da linha 010 entre 18:31 e 20:15, horário que se sobrepõe ao horário da viagem apontada no gabarito, entre 19:19 e 20:17.



### Carregar bibliotecas 

In [78]:
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.express as px

from datetime import timedelta, datetime

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve()

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'

for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"

### 1. Importar dados do gabarito e da solução

In [79]:
solucao = pd.read_excel('data/raw/amostra_linha_010_q1_2023_tratado.xlsx', sheet_name=0)  
solucao

,data,servico,sentido,id_veiculo,datetime_partida,datetime_chegada,extensao
0,2023-01-02,10,I,A72037,2023-01-02 06:07:57,2023-01-02 06:19:58,5505
1,2023-01-02,10,V,A72037,2023-01-02 06:32:00,2023-01-02 06:43:21,4902
2,2023-01-02,10,I,A72085,2023-01-02 06:35:11,2023-01-02 06:47:51,5505
3,2023-01-02,10,I,A72037,2023-01-02 07:00:04,2023-01-02 07:13:44,5505
4,2023-01-02,10,V,A72085,2023-01-02 07:05:29,2023-01-02 07:21:50,4902
...,...,...,...,...,...,...,...
729,2023-01-15,10,V,A72085,2023-01-15 19:18:23,2023-01-15 19:32:43,4902
730,2023-01-15,10,I,A72085,2023-01-15 19:42:45,2023-01-15 19:57:06,5505
731,2023-01-15,10,V,A72085,2023-01-15 20:08:57,2023-01-15 20:20:57,4902
732,2023-01-15,10,I,A72085,2023-01-15 20:32:29,2023-01-15 20:50:30,5505


In [80]:
gabarito = pd.read_excel('data/raw/amostra_linha_010_q1_2023_tratado.xlsx', sheet_name=1) 
gabarito

,data,servico,sentido,id_veiculo,datetime_partida,datetime_chegada,extensao
0,2023-01-02,10,I,A72037,2023-01-02 06:07:00,2023-01-02 06:22:00,5505
1,2023-01-02,10,V,A72037,2023-01-02 06:29:00,2023-01-02 06:56:00,4902
2,2023-01-02,10,I,A72085,2023-01-02 06:33:00,2023-01-02 06:52:00,5505
3,2023-01-02,10,I,A72037,2023-01-02 06:58:00,2023-01-02 07:16:00,5505
4,2023-01-02,10,V,A72085,2023-01-02 07:03:00,2023-01-02 07:29:00,4902
...,...,...,...,...,...,...,...
702,2023-01-15,10,I,A72085,2023-01-15 18:51:00,2023-01-15 19:14:00,5505
703,2023-01-15,10,V,A72085,2023-01-15 19:16:00,2023-01-15 19:38:00,4902
704,2023-01-15,10,I,A72085,2023-01-15 19:41:00,2023-01-15 19:59:00,5505
705,2023-01-15,10,V,A72085,2023-01-15 20:06:00,2023-01-15 20:28:00,4902


### 2. Tratar dados do gabarito

Remover os casos em que o mesmo veículo fez duas viagens no mesmo dia e horário.

In [81]:

gabarito = gabarito.sort_values(by=['id_veiculo', 'datetime_partida', 'datetime_chegada'])

# Filtrar linhas com base nas condições de sobreposição
mask = (
    gabarito.groupby('id_veiculo')['datetime_chegada'].shift(1).fillna(pd.Timestamp.min) <= gabarito['datetime_partida']
) & (
    gabarito['datetime_chegada'] <= gabarito.groupby('id_veiculo')['datetime_partida'].shift(-1).fillna(pd.Timestamp.max)
)
gabarito_tratado = gabarito[mask]

gabarito_tratado


,data,servico,sentido,id_veiculo,datetime_partida,datetime_chegada,extensao
0,2023-01-02,10,I,A72037,2023-01-02 06:07:00,2023-01-02 06:22:00,5505
1,2023-01-02,10,V,A72037,2023-01-02 06:29:00,2023-01-02 06:56:00,4902
3,2023-01-02,10,I,A72037,2023-01-02 06:58:00,2023-01-02 07:16:00,5505
5,2023-01-02,10,V,A72037,2023-01-02 07:28:00,2023-01-02 07:58:00,4902
8,2023-01-02,10,I,A72037,2023-01-02 08:02:00,2023-01-02 08:24:00,5505
...,...,...,...,...,...,...,...
100,2023-01-03,10,I,A72151,2023-01-03 17:50:00,2023-01-03 18:25:00,5505
102,2023-01-03,10,V,A72151,2023-01-03 18:27:00,2023-01-03 19:13:00,4902
104,2023-01-03,10,I,A72151,2023-01-03 19:19:00,2023-01-03 20:17:00,5505
105,2023-01-03,10,V,A72151,2023-01-03 20:18:00,2023-01-03 21:00:00,4902


In [82]:
# Verificar viagens do mesmo veículo no mesmo dia e horário
sobrepostos = gabarito.merge(gabarito_tratado, how='left', indicator=True)
sobrepostos = sobrepostos[sobrepostos['_merge'] == 'left_only']

sobrepostos


,data,servico,sentido,id_veiculo,datetime_partida,datetime_chegada,extensao,_merge
393,2023-01-03,10,V,A72085,2023-01-03 21:18:00,2023-01-03 21:37:00,4902,left_only
394,2023-01-03,10,I,A72085,2023-01-03 21:20:00,2023-01-03 21:36:00,5505,left_only


In [83]:
# Modificar gabarito_tratado para incluir uma das linhas do df sobrepostos
gabarito_tratado = pd.concat([gabarito_tratado, sobrepostos.iloc[[0]]])
gabarito_tratado = gabarito_tratado[['id_veiculo', 'datetime_partida', 'datetime_chegada']]
gabarito_tratado


,id_veiculo,datetime_partida,datetime_chegada
0,A72037,2023-01-02 06:07:00,2023-01-02 06:22:00
1,A72037,2023-01-02 06:29:00,2023-01-02 06:56:00
3,A72037,2023-01-02 06:58:00,2023-01-02 07:16:00
5,A72037,2023-01-02 07:28:00,2023-01-02 07:58:00
8,A72037,2023-01-02 08:02:00,2023-01-02 08:24:00
...,...,...,...
102,A72151,2023-01-03 18:27:00,2023-01-03 19:13:00
104,A72151,2023-01-03 19:19:00,2023-01-03 20:17:00
105,A72151,2023-01-03 20:18:00,2023-01-03 21:00:00
106,A72151,2023-01-03 21:03:00,2023-01-03 21:20:00


Na solução encontramos 734 viagens realizadas, contra 706 viagens realizadas no gabarito após o tratamento.

### 3. Comparar as viagens do gabarito com as viagens da solução

Para realizar a comparação entre os dados, foi considerado um desvio de 10 minutos para mais ou para menos no horário de partida.

In [84]:
# Calcular intervalos de 10 minutos
gabarito_tratado['inicio_intervalo'] = gabarito_tratado['datetime_partida'] - pd.Timedelta(seconds=600)
gabarito_tratado['fim_intervalo'] = gabarito_tratado['datetime_partida'] + pd.Timedelta(seconds=600)

In [85]:
# 1. Faça um left join usando 'id_veiculo' como chave
merged_df = gabarito_tratado.merge(solucao, on='id_veiculo', how='left', suffixes=('_gabarito', '_solucao'))

# 2. Filtrar as linhas com base nas condições de intervalo de tempo
resultado = merged_df[
    (merged_df['datetime_partida_solucao'] >= merged_df['inicio_intervalo']) & 
    (merged_df['datetime_partida_solucao'] <= merged_df['fim_intervalo'])
]

In [86]:
# 3. Remover linhas duplicadas
resultado = resultado.drop_duplicates()

# 4. Selecionar colunas específicas
resultado = resultado[['data', 'servico', 'sentido', 'id_veiculo', 'datetime_partida_gabarito', 
                       'datetime_chegada_gabarito', 'extensao', 'inicio_intervalo', 'fim_intervalo', 
                       'datetime_partida_solucao', 'datetime_chegada_solucao']]

resultado


,data,servico,sentido,id_veiculo,datetime_partida_gabarito,datetime_chegada_gabarito,extensao,inicio_intervalo,fim_intervalo,datetime_partida_solucao,datetime_chegada_solucao
0,2023-01-02,10,I,A72037,2023-01-02 06:07:00,2023-01-02 06:22:00,5505,2023-01-02 05:57:00,2023-01-02 06:17:00,2023-01-02 06:07:57,2023-01-02 06:19:58
259,2023-01-02,10,V,A72037,2023-01-02 06:29:00,2023-01-02 06:56:00,4902,2023-01-02 06:19:00,2023-01-02 06:39:00,2023-01-02 06:32:00,2023-01-02 06:43:21
518,2023-01-02,10,I,A72037,2023-01-02 06:58:00,2023-01-02 07:16:00,5505,2023-01-02 06:48:00,2023-01-02 07:08:00,2023-01-02 07:00:04,2023-01-02 07:13:44
777,2023-01-02,10,V,A72037,2023-01-02 07:28:00,2023-01-02 07:58:00,4902,2023-01-02 07:18:00,2023-01-02 07:38:00,2023-01-02 07:31:27,2023-01-02 07:47:47
1036,2023-01-02,10,I,A72037,2023-01-02 08:02:00,2023-01-02 08:24:00,5505,2023-01-02 07:52:00,2023-01-02 08:12:00,2023-01-02 08:03:43,2023-01-02 08:22:04
...,...,...,...,...,...,...,...,...,...,...,...
208428,2023-01-03,10,I,A72151,2023-01-03 17:50:00,2023-01-03 18:25:00,5505,2023-01-03 17:40:00,2023-01-03 18:00:00,2023-01-03 17:51:34,2023-01-03 18:22:35
208436,2023-01-03,10,V,A72151,2023-01-03 18:27:00,2023-01-03 19:13:00,4902,2023-01-03 18:17:00,2023-01-03 18:37:00,2023-01-03 18:30:36,2023-01-03 18:59:52
208452,2023-01-03,10,V,A72151,2023-01-03 20:18:00,2023-01-03 21:00:00,4902,2023-01-03 20:08:00,2023-01-03 20:28:00,2023-01-03 20:21:27,2023-01-03 20:45:28
208460,2023-01-03,10,I,A72151,2023-01-03 21:03:00,2023-01-03 21:20:00,5505,2023-01-03 20:53:00,2023-01-03 21:13:00,2023-01-03 21:04:37,2023-01-03 21:18:38


Foram encontradas 699 das 706 viagens apresentadas no gabarito (94,75% das viagens do gabarito).

### 4. Identificar as viagens que não foram encontradas na solução e que estão no gabarito

In [105]:
tabela = gabarito_tratado.merge(resultado, how = 'left', 
                                left_on=['id_veiculo', 'datetime_partida'], 
                                right_on=['id_veiculo', 'datetime_partida_gabarito'])

linhas_nan = tabela[tabela['sentido'].isna()]
linhas_nan

,id_veiculo,datetime_partida,datetime_chegada,inicio_intervalo_x,fim_intervalo_x,data,servico,sentido,datetime_partida_gabarito,datetime_chegada_gabarito,extensao,inicio_intervalo_y,fim_intervalo_y,datetime_partida_solucao,datetime_chegada_solucao
51,A72037,2023-01-04 06:00:00,2023-01-04 06:16:00,2023-01-04 05:50:00,2023-01-04 06:10:00,NaT,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaT,NaT
52,A72037,2023-01-04 06:28:00,2023-01-04 07:01:00,2023-01-04 06:18:00,2023-01-04 06:38:00,NaT,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaT,NaT
53,A72037,2023-01-04 07:03:00,2023-01-04 07:26:00,2023-01-04 06:53:00,2023-01-04 07:13:00,NaT,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaT,NaT
54,A72037,2023-01-04 07:29:00,2023-01-04 07:53:00,2023-01-04 07:19:00,2023-01-04 07:39:00,NaT,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaT,NaT
394,A72085,2023-01-04 06:30:00,2023-01-04 06:50:00,2023-01-04 06:20:00,2023-01-04 06:40:00,NaT,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaT,NaT
672,A72085,2023-01-15 07:20:00,2023-01-15 07:55:00,2023-01-15 07:10:00,2023-01-15 07:30:00,NaT,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaT,NaT
702,A72151,2023-01-03 19:19:00,2023-01-03 20:17:00,2023-01-03 19:09:00,2023-01-03 19:29:00,NaT,NaN,NaN,NaT,NaT,NaN,NaT,NaT,NaT,NaT


Sete viagens que constam no gabarito não foram encontradas na solução e foram realizadas por três veículos diferentes. 

1) Quatro viagens do veículo A72037 no início do dia 04/01/2023

In [99]:
tabela['datetime_partida'] = pd.to_datetime(tabela['datetime_partida'])

tabela_A72037 = tabela[(tabela['id_veiculo'] == 'A72037') & (tabela['datetime_partida'].dt.date == pd.Timestamp('2023-01-04').date())]

tabela_A72037 = tabela_A72037[['id_veiculo', 'datetime_partida','datetime_chegada','datetime_partida_solucao','datetime_chegada_solucao']]
tabela_A72037

,id_veiculo,datetime_partida,datetime_chegada,datetime_partida_solucao,datetime_chegada_solucao
51,A72037,2023-01-04 06:00:00,2023-01-04 06:16:00,NaT,NaT
52,A72037,2023-01-04 06:28:00,2023-01-04 07:01:00,NaT,NaT
53,A72037,2023-01-04 07:03:00,2023-01-04 07:26:00,NaT,NaT
54,A72037,2023-01-04 07:29:00,2023-01-04 07:53:00,NaT,NaT
55,A72037,2023-01-04 08:01:00,2023-01-04 08:24:00,2023-01-04 08:03:10,2023-01-04 08:22:11
56,A72037,2023-01-04 08:33:00,2023-01-04 09:03:00,2023-01-04 08:38:58,2023-01-04 08:57:58
57,A72037,2023-01-04 09:07:00,2023-01-04 09:30:00,2023-01-04 09:09:59,2023-01-04 09:28:19
58,A72037,2023-01-04 09:32:00,2023-01-04 10:01:00,2023-01-04 09:38:20,2023-01-04 09:56:40
59,A72037,2023-01-04 10:05:00,2023-01-04 10:33:00,2023-01-04 10:08:41,2023-01-04 10:28:42
60,A72037,2023-01-04 10:34:00,2023-01-04 11:04:00,2023-01-04 10:40:02,2023-01-04 10:59:22


Ao verificar a tabela de conformidade, também não foram encontradas as viagens antes de 08:00.

In [106]:
q = f"""
    SELECT
       *
     FROM
       `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
      WHERE
       data = "2023-01-04"
       AND id_veiculo = 'A72037'
"""

registros_conformidade_A72037 = bd.read_sql(q)

Downloading: 100%|██████████| 12/12 [00:00<00:00, 26.32rows/s]


In [110]:
registros_conformidade_A72037 = registros_conformidade_A72037[['id_veiculo','datetime_partida','datetime_chegada']]
registros_conformidade_A72037.sort_values(by='datetime_partida')

,id_veiculo,datetime_partida,datetime_chegada
5,A72037,2023-01-04 08:39:58,2023-01-04 09:28:19
4,A72037,2023-01-04 09:39:40,2023-01-04 10:28:42
8,A72037,2023-01-04 10:40:42,2023-01-04 11:32:24
1,A72037,2023-01-04 11:41:44,2023-01-04 12:26:25
3,A72037,2023-01-04 12:35:53,2023-01-04 13:22:54
7,A72037,2023-01-04 13:34:49,2023-01-04 14:36:44
10,A72037,2023-01-04 14:44:45,2023-01-04 15:46:35
11,A72037,2023-01-04 15:56:36,2023-01-04 16:56:53
6,A72037,2023-01-04 17:05:34,2023-01-04 18:07:08
9,A72037,2023-01-04 18:16:48,2023-01-04 19:11:35


In [140]:
q = f"""
SELECT
  *
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = "2023-01-04"
  AND id_veiculo = 'A72037'
  AND flag_em_operacao = true
"""

registros_gps_A72037 = bd.read_sql(q)

Downloading: 100%|██████████| 2577/2577 [00:00<00:00, 2975.75rows/s]


In [143]:
registros_gps_A72037 = registros_gps_A72037[['id_veiculo', 'hora','servico']]
registros_gps_A72037.sort_values(by='hora')

,id_veiculo,hora,servico
1654,A72037,06:47:25,007
2364,A72037,06:47:45,007
1355,A72037,06:48:06,007
1964,A72037,06:48:26,007
1403,A72037,06:48:46,007
...,...,...,...
288,A72037,21:38:04,010
1547,A72037,21:38:24,010
1785,A72037,21:38:44,010
1341,A72037,21:39:04,010


Ao verificar a tabela com os sinais de GPS, entre 06:47 e 07:45, intervalo que é compatível com duas das quatro viagens não identificadas, foram identificados sinais do veículo operando no serviço 007, e não no serviço 010. Ou seja, das quatro viagens, duas não foram identificadas e as outras duas foram realizadas em outro serviço.

2) Duas viagens do veículo A72085: uma no dia 04/01/2023 e outra no dia 15/01/2023

2.1) Viagem do veículo A72085 no dia 04/01/2023

In [102]:
tabela_A72085_04_01 = tabela[(tabela['id_veiculo'] == 'A72085') & (tabela['datetime_partida'].dt.date == pd.Timestamp('2023-01-04').date())]

tabela_A72085_04_01 = tabela_A72085_04_01[['id_veiculo', 'datetime_partida','datetime_chegada','datetime_partida_solucao','datetime_chegada_solucao']]
tabela_A72085_04_01

,id_veiculo,datetime_partida,datetime_chegada,datetime_partida_solucao,datetime_chegada_solucao
394,A72085,2023-01-04 06:30:00,2023-01-04 06:50:00,NaT,NaT
395,A72085,2023-01-04 06:59:00,2023-01-04 07:28:00,2023-01-04 07:01:53,2023-01-04 07:14:14
396,A72085,2023-01-04 07:30:00,2023-01-04 07:50:00,2023-01-04 07:31:57,2023-01-04 07:48:17
397,A72085,2023-01-04 07:59:00,2023-01-04 08:29:00,2023-01-04 08:03:39,2023-01-04 08:22:40
398,A72085,2023-01-04 08:32:00,2023-01-04 09:00:00,2023-01-04 08:34:00,2023-01-04 08:56:41
399,A72085,2023-01-04 09:03:00,2023-01-04 09:31:00,2023-01-04 09:06:01,2023-01-04 09:22:02
400,A72085,2023-01-04 09:34:00,2023-01-04 10:04:00,2023-01-04 09:37:39,2023-01-04 10:02:20
401,A72085,2023-01-04 10:07:00,2023-01-04 10:33:00,2023-01-04 10:10:20,2023-01-04 10:28:01
402,A72085,2023-01-04 10:35:00,2023-01-04 11:04:00,2023-01-04 10:38:01,2023-01-04 11:01:02
403,A72085,2023-01-04 11:06:00,2023-01-04 11:27:00,2023-01-04 11:10:42,2023-01-04 11:25:03


A viagem não foi encontrada na tabela de conformidade

In [121]:
q = f"""
    SELECT
       *
     FROM
       `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
      WHERE
       data = "2023-01-04"
       AND id_veiculo = 'A72085'
"""

registros_conformidade_A72085_04 = bd.read_sql(q)

Downloading: 100%|██████████| 14/14 [00:00<00:00, 39.55rows/s]


In [122]:
registros_conformidade_A72085_04 = registros_conformidade_A72085_04[['id_veiculo', 'datetime_partida','datetime_chegada']]
registros_conformidade_A72085_04.sort_values(by='datetime_partida')

,id_veiculo,datetime_partida,datetime_chegada
2,A72085,2023-01-04 07:02:13,2023-01-04 07:48:17
8,A72085,2023-01-04 08:04:19,2023-01-04 08:56:41
11,A72085,2023-01-04 09:06:01,2023-01-04 10:02:20
6,A72085,2023-01-04 10:10:40,2023-01-04 11:01:02
5,A72085,2023-01-04 11:11:22,2023-01-04 11:56:04
9,A72085,2023-01-04 12:06:44,2023-01-04 13:00:25
7,A72085,2023-01-04 13:10:06,2023-01-04 14:02:27
10,A72085,2023-01-04 14:10:27,2023-01-04 15:00:49
13,A72085,2023-01-04 15:09:50,2023-01-04 16:05:22
12,A72085,2023-01-04 16:14:02,2023-01-04 17:05:00


In [137]:
q = f"""
SELECT
  *
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = "2023-01-04"
  AND id_veiculo = 'A72085'
  AND flag_em_operacao = true
"""

registros_gps_A72085_04 = bd.read_sql(q)

Downloading: 100%|██████████| 2538/2538 [00:00<00:00, 2920.60rows/s]


In [139]:
registros_gps_A72085_04 = registros_gps_A72085_04[['id_veiculo', 'hora','servico']]
registros_gps_A72085_04.sort_values(by='hora')

,id_veiculo,hora,servico
2359,A72085,06:47:41,010
1745,A72085,06:48:01,010
1718,A72085,06:48:21,010
382,A72085,06:48:41,010
2155,A72085,06:49:01,010
...,...,...,...
2294,A72085,21:59:27,010
1190,A72085,21:59:47,010
1070,A72085,22:00:07,010
125,A72085,22:00:27,010


Foi identificado sinal de GPS em operação para o veículo A72085 desde 06:47, horário próximo ao horário do final da viagem não identificada (06:50). Os sinais de GPS não contemplaram a duração total da viagem.

2.2) Viagem do veículo A72085 no dia 15/01/2023

In [127]:
tabela_A72085_04_15 = tabela[(tabela['id_veiculo'] == 'A72085') & (tabela['datetime_partida'].dt.date == pd.Timestamp('2023-01-15').date())]

tabela_A72085_04_15 = tabela_A72085_04_15[['id_veiculo', 'datetime_partida','datetime_chegada','datetime_partida_solucao','datetime_chegada_solucao']]
tabela_A72085_04_15

,id_veiculo,datetime_partida,datetime_chegada,datetime_partida_solucao,datetime_chegada_solucao
669,A72085,2023-01-15 06:36:00,2023-01-15 06:52:00,2023-01-15 06:37:54,2023-01-15 06:50:34
670,A72085,2023-01-15 06:52:00,2023-01-15 07:04:00,2023-01-15 06:54:54,2023-01-15 07:03:34
671,A72085,2023-01-15 07:05:00,2023-01-15 07:19:00,2023-01-15 07:06:55,2023-01-15 07:17:15
672,A72085,2023-01-15 07:20:00,2023-01-15 07:55:00,NaT,NaT
673,A72085,2023-01-15 08:01:00,2023-01-15 08:17:00,2023-01-15 08:03:32,2023-01-15 08:15:12
674,A72085,2023-01-15 08:27:00,2023-01-15 09:02:00,2023-01-15 08:31:13,2023-01-15 08:43:53
675,A72085,2023-01-15 09:06:00,2023-01-15 09:24:00,2023-01-15 09:08:37,2023-01-15 09:20:17
676,A72085,2023-01-15 09:32:00,2023-01-15 09:52:00,2023-01-15 09:35:39,2023-01-15 09:46:39
677,A72085,2023-01-15 10:02:00,2023-01-15 10:19:00,2023-01-15 10:04:37,2023-01-15 10:16:37
678,A72085,2023-01-15 10:25:00,2023-01-15 10:49:00,2023-01-15 10:30:31,2023-01-15 10:44:51


In [131]:
q = f"""
    SELECT
       *
     FROM
       `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
      WHERE
       data = "2023-01-15"
       AND id_veiculo = 'A72085'
"""

registros_conformidade_A72085_15 = bd.read_sql(q)

Downloading: 0rows [00:00, ?rows/s]


In [132]:
registros_conformidade_A72085_15 

,id_viagem,data,id_empresa,id_veiculo,servico_informado,servico_realizado,distancia_planejada,sentido,datetime_partida,datetime_chegada,...,n_registros_start,n_registros_end,n_registros_out,n_registros_total,n_registros_minuto,n_registros_shape,perc_conformidade_shape,perc_conformidade_distancia,perc_conformidade_registros,versao_modelo


Não foram encontradas viagens para o veículo A72085 no dia 15-01-2023 na tabela de viagem_conformidade. 

In [148]:
q = f"""
SELECT
  *
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = "2023-01-15"
  AND id_veiculo = 'A72085'
  AND flag_em_operacao = true
"""

registros_gps_A72085_15 = bd.read_sql(q)

Downloading: 100%|██████████| 2455/2455 [00:00<00:00, 2746.05rows/s]


In [149]:
registros_gps_A72085_15 = registros_gps_A72085_15[['id_veiculo', 'hora','servico']]

hora_inicial = datetime.strptime('07:20', '%H:%M').time()
hora_final = datetime.strptime('07:55', '%H:%M').time()

registros_gps_A72085_15 = registros_gps_A72085_15[(registros_gps_A72085_15['hora'] >= hora_inicial) & (registros_gps_A72085_15['hora'] <= hora_final)]

registros_gps_A72085_15.sort_values(by='hora')

,id_veiculo,hora,servico
657,A72085,07:20:03,010
1782,A72085,07:20:23,010
2154,A72085,07:20:43,010
419,A72085,07:21:03,010
369,A72085,07:21:23,010
...,...,...,...
259,A72085,07:53:22,010
1001,A72085,07:53:42,010
520,A72085,07:54:02,010
864,A72085,07:54:22,010


Foram encontrados sinais de GPS para o mesmo veículo, dia e serviço.

3) Uma viagem do veículo A72151 no dia 03/01/2023

In [150]:
tabela_A72151 = tabela[(tabela['id_veiculo'] == 'A72151') & (tabela['datetime_partida'].dt.date == pd.Timestamp('2023-01-03').date())]
tabela_A72151 = tabela_A72151[['id_veiculo', 'datetime_partida','datetime_chegada','datetime_partida_solucao','datetime_chegada_solucao']]
tabela_A72151

,id_veiculo,datetime_partida,datetime_chegada,datetime_partida_solucao,datetime_chegada_solucao
700,A72151,2023-01-03 17:50:00,2023-01-03 18:25:00,2023-01-03 17:51:34,2023-01-03 18:22:35
701,A72151,2023-01-03 18:27:00,2023-01-03 19:13:00,2023-01-03 18:30:36,2023-01-03 18:59:52
702,A72151,2023-01-03 19:19:00,2023-01-03 20:17:00,NaT,NaT
703,A72151,2023-01-03 20:18:00,2023-01-03 21:00:00,2023-01-03 20:21:27,2023-01-03 20:45:28
704,A72151,2023-01-03 21:03:00,2023-01-03 21:20:00,2023-01-03 21:04:37,2023-01-03 21:18:38


Checar a tabela de conformidade

In [151]:
q = f"""
    SELECT
       *
     FROM
       `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
      WHERE
       data = "2023-01-03"
       AND id_veiculo = 'A72151'
"""

registros_conformidade_A72151 = bd.read_sql(q)

Downloading: 100%|██████████| 17/17 [00:00<00:00, 47.75rows/s]


In [153]:
registros_conformidade_A72151.sort_values(by = 'datetime_partida')

,id_viagem,data,id_empresa,id_veiculo,servico_informado,servico_realizado,distancia_planejada,sentido,datetime_partida,datetime_chegada,...,n_registros_start,n_registros_end,n_registros_out,n_registros_total,n_registros_minuto,n_registros_shape,perc_conformidade_shape,perc_conformidade_distancia,perc_conformidade_registros,versao_modelo
0,A72151-006-I-fm9l-20230103060937,2023-01-03,721,A72151,006,006,8.427,I,2023-01-03 06:09:37,2023-01-03 06:30:36,...,1,1,0,84,22,84,100.0,85.46,100.00,144ea938e197b43767a3ad3a661bf7e8aaae4656
3,A72151-006-V-wu1v-20230103063636,2023-01-03,721,A72151,006,006,8.885,V,2023-01-03 06:36:36,2023-01-03 06:59:37,...,1,1,0,93,24,93,100.0,89.12,100.00,144ea938e197b43767a3ad3a661bf7e8aaae4656
9,A72151-006-I-fm9l-20230103071644,2023-01-03,721,A72151,006,006,8.427,I,2023-01-03 07:16:44,2023-01-03 07:43:17,...,1,1,0,104,28,104,100.0,87.04,100.00,144ea938e197b43767a3ad3a661bf7e8aaae4656
10,A72151-006-V-wu1v-20230103075032,2023-01-03,721,A72151,006,006,8.885,V,2023-01-03 07:50:32,2023-01-03 08:17:38,...,1,1,0,110,28,110,100.0,90.98,100.00,144ea938e197b43767a3ad3a661bf7e8aaae4656
1,A72151-006-I-fm9l-20230103082749,2023-01-03,721,A72151,006,006,8.427,I,2023-01-03 08:27:49,2023-01-03 08:50:50,...,1,1,0,93,24,93,100.0,86.69,100.00,144ea938e197b43767a3ad3a661bf7e8aaae4656
15,A72151-006-V-wu1v-20230103085921,2023-01-03,721,A72151,006,006,8.885,V,2023-01-03 08:59:21,2023-01-03 09:30:06,...,1,1,0,124,32,124,100.0,90.68,100.00,144ea938e197b43767a3ad3a661bf7e8aaae4656
5,A72151-006-I-fm9l-20230103094518,2023-01-03,721,A72151,006,006,8.427,I,2023-01-03 09:45:18,2023-01-03 10:09:08,...,1,1,0,97,25,97,100.0,85.53,100.00,144ea938e197b43767a3ad3a661bf7e8aaae4656
14,A72151-006-V-wu1v-20230103101638,2023-01-03,721,A72151,006,006,8.885,V,2023-01-03 10:16:38,2023-01-03 10:47:10,...,1,1,0,124,32,124,100.0,89.03,100.00,144ea938e197b43767a3ad3a661bf7e8aaae4656
6,A72151-006-I-fm9l-20230103110541,2023-01-03,721,A72151,006,006,8.427,I,2023-01-03 11:05:41,2023-01-03 11:30:27,...,1,1,0,100,26,100,100.0,86.26,100.00,144ea938e197b43767a3ad3a661bf7e8aaae4656
2,A72151-006-V-wu1v-20230103114012,2023-01-03,721,A72151,006,006,8.885,V,2023-01-03 11:40:12,2023-01-03 12:03:13,...,1,1,0,93,24,93,100.0,87.00,100.00,144ea938e197b43767a3ad3a661bf7e8aaae4656


Foi encontrada uma viagem na linha 010 entre 18:31 e 20:15, horário que se sobrepõe ao horário da viagem apontada no gabarito, entre 19:19 e 20:17.

5. Viagens que foram encontradas na solução, mas que não foram encontradas no gabarito.

Trinta e cinco viagens que constam na solução não foram encontradas no gabarito.

In [154]:
tabela = solucao.merge(resultado, how = 'left', 
                                left_on=['id_veiculo', 'datetime_partida'], 
                                right_on=['id_veiculo', 'datetime_partida_solucao'])

tabela

linhas_nan = tabela[tabela['sentido_y'].isna()]
linhas_nan
len(linhas_nan)

35

### 6. Conclusão

Ao considerar um intervalo de 10 minutos para mais ou para menos entre os horários de partida, foram encontradas 699 das 706 viagens apresentadas no gabarito (94,75% das viagens do gabarito).

Ao analisar as sete viagens não identificadas na solução acima, concluiu-se que:

- Não foram encontrados dados das quatro viagens do veículo A72037 na linha 010, mas foram encontrados sinais de GPS em horários compatíveis com duas das quatro viagens, operando na linha 007.

- No caso do veículo A72085, na viagem do dia 15/01/2023 foram encontrados sinais de GPS para o mesmo veículo, dia e serviço (mas não foram encontrados dados para este dia na tabela de conformidade). Quanto a viagem do dia 04/01/2023, foram identificados sinais de GPS apenas em horário próximo ao término da viagem.

- Para o veículo A72151, foi encontrada uma viagem da linha 010 entre 18:31 e 20:15, horário que se sobrepõe ao horário da viagem apontada no gabarito, entre 19:19 e 20:17.